# Coronavirus Prevalence by US County
By: Steven E. Davis


In [1]:
# Using Pandas data analysis tool
import pandas as pd
# Get Pandas version
pd.__version__

'0.25.1'

In [2]:
# Set Jupyter qtconsole options ---- 
# Display Rows: (default to 10 rows)
pd.set_option('display.max_rows', 10)
# None displays ALL rows (could be lots of rows so be careful!)
#pd.set_option('display.max_rows', None)

# 2019 Population Data Estimates by US County
Source:  
* The United States Census Bureau: 
https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/

In [3]:
# Import population for each US Zip Code
fr_est2019_pop_all = pd.read_csv('co-est2019-alldata_sd.csv')
# Quick Look
#fr_est2019_pop_all

In [4]:
# Select Columns:
# state, city (which is really state/city/county), and populations for 2010 and 2019
fr_est2019_pop_cols = fr_est2019_pop_all.loc[:, ['STNAME', 'CTYNAME', 'CENSUS2010POP', 'POPESTIMATE2019']]
#fr_est2019_pop_cols                

In [5]:
# Rename columns to make data easier to read
fr_est2019_pop_renam = fr_est2019_pop_cols.rename(columns = {
    'STNAME':'state_name', 
    'CTYNAME':'county', 
    'CENSUS2010POP':'pop2010', 
    'POPESTIMATE2019':'pop2019'})
#fr_est2019_pop_renam

In [6]:
# Exclude rows that indicate total state population () where state == county
# Exception:  Need to leave the 'District of Columbia' in the data set since it is reported like a county.
fr_est2019_pop_county = fr_est2019_pop_renam.query('state_name != county or state_name == \"District of Columbia\"').copy()
#fr_est2019_pop_county

In [7]:
# Filter out county suffixes ('County', 'Parish', etc.) so county names will exactly match the virus data
fr_est2019_pop_county['county'] = fr_est2019_pop_county['county'].replace(to_replace=r'\b County\b',value='', regex=True)
fr_est2019_pop_county['county'] = fr_est2019_pop_county['county'].replace(to_replace=r'\b Municipality\b',value='', regex=True)
fr_est2019_pop_county['county'] = fr_est2019_pop_county['county'].replace(to_replace=r'\b Parish\b',value='', regex=True)

# Sort by State then County
fr_est2019_pop_county.sort_values(by=['state_name', 'county'], inplace=True)

#fr_est2019_pop_county

In [8]:
pd.set_option('display.max_rows', 10)

# Hardcoded Exceptions:
# Several exceptions are noted in the virus data in order to exactly match the county names:

# New York City, NY:
# All cases for the five boroughs of New York City (New York, Kings, Queens, Bronx and Richmond counties) 
# are assigned to a single area called New York City.
# Population Estimate for 2010 (add the 5 boroughs) = 8,190,209
# Population Estimate for 2019 (add the 5 boroughs) = 8,336,817
row_df = pd.DataFrame({'state_name': ['New York'], 'county':['New York City'], 'pop2010':[8190209], 'pop2019':[8336817]})
fr_est2019_pop_county = fr_est2019_pop_county.append(row_df, ignore_index=True).copy()

# Kansas City, MO:
# Four counties (Cass, Clay, Jackson and Platte) overlap the municipality of Kansas City, Mo. 
# The cases and deaths that we show for these four counties are only for the portions exclusive of Kansas City. 
# Cases and deaths for Kansas City are reported as their own line.
row_df = pd.DataFrame({'state_name': ['Missouri'], 'county':['Kansas City'], 'pop2010':[459787], 'pop2019':[491918]})
fr_est2019_pop_county = fr_est2019_pop_county.append(row_df, ignore_index=True).copy()

#pd.set_option('display.max_rows', None)
fr_est2019_pop_county


,state_name,county,pop2010,pop2019
0,Alabama,Autauga,54571,55869
1,Alabama,Baldwin,182265,223234
2,Alabama,Barbour,27457,24686
3,Alabama,Bibb,22915,22394
4,Alabama,Blount,57322,57826
...,...,...,...,...
3140,Wyoming,Uinta,21118,20226
3141,Wyoming,Washakie,8533,7805
3142,Wyoming,Weston,7208,6927
3143,New York,New York City,8190209,8336817


# Corona Virus Cases by US County
Data Source:
* New Yourk Times ongoing repository of data on coronavirus cases and deaths in the U.S.:
https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv

In [9]:
pd.set_option('display.max_rows', 10)

# Import Corona Virus Cases by US County

# OPTION 1: Read from local file system:
#fr_cv_county = pd.read_csv('us-counties.csv')

# OPTION 2 (preferred): Read directly from GitHub:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
fr_cv_county = pd.read_csv(url)

# Rename the state column so we can later include the state abbreviation
fr_cv_county.rename(columns={'state':'state_name'}, inplace=True)

#pd.set_option('display.max_rows', None)
#fr_cv_county

In [10]:
# Define a state name to abbreviation map/dictionary
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
#abbrev_us_state = pd.DataFrame(us_state_abbrev.items(), columns=['state_name', 'state'])

In [11]:
# Map the state full name to the two character abbreviation
fr_cv_county['state'] = fr_cv_county['state_name'].map(us_state_abbrev)

# Sort data by state name then county (then date)
fr_cv_county.sort_values(by=['state_name', 'county', 'date'], inplace=True)

#fr_cv_county

In [12]:
# Remove data for 'Unknown' counties
# Wwe are just interested in data associated with a named county
fr_cv_county_clean = fr_cv_county.query('county != \"Unknown\"').copy()
#fr_cv_county_clean

In [13]:
pd.set_option('display.max_rows', 10)

# Remove Data for counties that have been combined
# These started sneeking in on 04/04 with zero data.
indexNames = fr_cv_county_clean[(fr_cv_county_clean['county'] == 'Bronx') & 
                                (fr_cv_county_clean['state_name'] == 'New York')].index
fr_cv_county_clean.drop(indexNames, inplace=True)
indexNames = fr_cv_county_clean[(fr_cv_county_clean['county'] == 'Kings') & 
                                (fr_cv_county_clean['state_name'] == 'New York')].index
fr_cv_county_clean.drop(indexNames, inplace=True)
indexNames = fr_cv_county_clean[(fr_cv_county_clean['county'] == 'Queens') & 
                                (fr_cv_county_clean['state_name'] == 'New York')].index
fr_cv_county_clean.drop(indexNames, inplace=True)
indexNames = fr_cv_county_clean[(fr_cv_county_clean['county'] == 'Richmond') & 
                                (fr_cv_county_clean['state_name'] == 'New York')].index
fr_cv_county_clean.drop(indexNames, inplace=True)

#pd.set_option('display.max_rows', None)
fr_cv_county_clean

,date,county,state_name,fips,cases,deaths,state
9481,2020-03-24,Autauga,Alabama,1001.0,1,0,AL
10839,2020-03-25,Autauga,Alabama,1001.0,4,0,AL
12377,2020-03-26,Autauga,Alabama,1001.0,6,0,AL
14046,2020-03-27,Autauga,Alabama,1001.0,6,0,AL
15836,2020-03-28,Autauga,Alabama,1001.0,6,0,AL
...,...,...,...,...,...,...,...
28488,2020-04-02,Washakie,Wyoming,56043.0,2,0,WY
30845,2020-04-03,Washakie,Wyoming,56043.0,2,0,WY
33255,2020-04-04,Washakie,Wyoming,56043.0,2,0,WY
35699,2020-04-05,Washakie,Wyoming,56043.0,4,0,WY


### >>  Merge Population Data with Virus Data

In [14]:
pd.set_option('display.max_rows', 10)

# Merge the corona data with the county population
# Doing a left join here to include all rows in the virus data and matching them with appropriate county data
fr_cv_pop_county = pd.merge(fr_cv_county_clean, fr_est2019_pop_county, on=['state_name','county'], how='left').copy()

#pd.set_option('display.max_rows', None)
#fr_cv_pop_county

In [15]:
# Quality Checks: (output should be empty if check passes)
pd.set_option('display.max_rows', 10)

# Check to see if there are any unmatched rows (both pop2010 and pop2019 will be NaN)
fr_null_set = fr_cv_pop_county[fr_cv_pop_county['pop2010'].isnull()]

#pd.set_option('display.max_rows', None)
fr_null_set

,date,county,state_name,fips,cases,deaths,state,pop2010,pop2019


In [16]:
pd.set_option('display.max_rows', 10)

# Calculate Prevalence (as a percent)
# ((number of cases)/(population)) * 100
# For 2019 Population Estimate only:

# 2019 Prevalence:
fr_cv_pop_county['prev2019'] = fr_cv_pop_county['cases'] * 100 / fr_cv_pop_county['pop2019']

# Print entire data set to do a visual check
#pd.set_option('display.max_rows', None)
fr_cv_pop_county

,date,county,state_name,fips,cases,deaths,state,pop2010,pop2019,prev2019
0,2020-03-24,Autauga,Alabama,1001.0,1,0,AL,54571,55869,0.001790
1,2020-03-25,Autauga,Alabama,1001.0,4,0,AL,54571,55869,0.007160
2,2020-03-26,Autauga,Alabama,1001.0,6,0,AL,54571,55869,0.010739
3,2020-03-27,Autauga,Alabama,1001.0,6,0,AL,54571,55869,0.010739
4,2020-03-28,Autauga,Alabama,1001.0,6,0,AL,54571,55869,0.010739
...,...,...,...,...,...,...,...,...,...,...
37721,2020-04-02,Washakie,Wyoming,56043.0,2,0,WY,8533,7805,0.025625
37722,2020-04-03,Washakie,Wyoming,56043.0,2,0,WY,8533,7805,0.025625
37723,2020-04-04,Washakie,Wyoming,56043.0,2,0,WY,8533,7805,0.025625
37724,2020-04-05,Washakie,Wyoming,56043.0,4,0,WY,8533,7805,0.051249


In [17]:
# Save Data: cv_pop_county.csv
fr_cv_pop_county.to_csv('cv_pop_county.csv', index=False)

In [18]:
pd.set_option('display.max_rows', 10)

# Most Prevalent
fr_cv_pop_county.sort_values(by=['prev2019'], ascending=False, inplace=True)

#pd.set_option('display.max_rows', None)
fr_cv_pop_county

,date,county,state_name,fips,cases,deaths,state,pop2010,pop2019,prev2019
8616,2020-04-06,Blaine,Idaho,16013.0,423,2,ID,21376,23021,1.837453
8615,2020-04-05,Blaine,Idaho,16013.0,410,2,ID,21376,23021,1.780983
8614,2020-04-04,Blaine,Idaho,16013.0,410,2,ID,21376,23021,1.780983
8613,2020-04-03,Blaine,Idaho,16013.0,405,2,ID,21376,23021,1.759263
23005,2020-04-06,Rockland,New York,36087.0,5703,76,NY,311687,325789,1.750520
...,...,...,...,...,...,...,...,...,...,...
2615,2020-02-04,Los Angeles,California,6037.0,1,0,CA,9818605,10039107,0.000010
31601,2020-04-06,Crosby,Texas,48107.0,0,0,TX,6059,5737,0.000000
32016,2020-04-06,Hansford,Texas,48195.0,0,0,TX,5613,5399,0.000000
22742,2020-04-06,New York,New York,36061.0,0,0,NY,1585873,1628706,0.000000
